In [1]:
import sys
import arcpy
import os

In [2]:
# sprawdzanie rozszerzeń
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")
# ustawienia środowiska pracy
arcpy.env.workspace = r'D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\swieradow.gdb'
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput=True
arcpy.env.cellSize= 1
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps()[0]

In [3]:
#ustawianie sciezek i parametrow charakterystycznych dla danego obszaru
wynikowe_path = "D:\\OneDrive - Politechnika Warszawska\\Anazlizy\\Swieradow\\swieradow\\wynikowe\\"
gdb_path = r'D:\\OneDrive - Politechnika Warszawska\\Anazlizy\\Swieradow\\swieradow\\swieradow.gdb'
przedrostek_bdot = "PL.PZGiK.337.0210__OT_"
BDOT_path = 'D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\PowiatLubanskiBDOT'
#przejsciowe_path = r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\przejsciowe\\"
nazwa_gminy = "'Świeradów-Zdrój'"

#przypisanie do zmiennych sciezek do danych do ktorych mamy gotowe warstwy
hotele = r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\wynikowe\hotele"
gleby = r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\Gleby\gleby_swieradow_zdroj\gleby_gm_swieradow.shp"
gazociag = r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\Dane_gazociag\gazociag.shp"
dzialki = r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\dzialki\dzialki.shp"
NMT = "swieradowPolaczony.tif"

Wgranie danych

In [12]:
#tablica ktora bedzie przechowywala nazwy warstw pokrycia terenu ktore wystepuja w porjekcie
PT = []

#przeiterowanie sie po katalogu bdotu i dodanie wszystkich warstw pokrycia terenu
for files in os.listdir(BDOT_path):
    if files.endswith('.shp') and files.startswith(przedrostek_bdot + "PT"):
        print(files)
        PT.append(files[0:-4])
        path = BDOT_path +"\\" + files
        map.addDataFromPath(path)
    else:
        continue

#polaczenie dodatnych warstw pokrycia, z wykorzystaniem tablicy do ktorej przy iteracji po katalogu dodawalismy nazwy plikow pokrycia
#i zapisanie wyniku nastepnie dodanie do mapy
pokrycie_terenu = arcpy.management.Merge(PT, wynikowe_path + 'pokrycieTerenu.shp')
map.addDataFromPath(wynikowe_path + 'pokrycieTerenu.shp')

#ustalenie jakie warstwy bdota chcemy dodac do projektu nastepnie przygotowanie ich, aby miec pelne nazwy plikow razem z przedrostkiem dla danego terenu
bdot_layers = ['ADMS_A', 'SKDR_L', 'PTWP_A','SWRS_L', 'BUBD_A', 'PTLZ_A']
bdot_layers_1 = [przedrostek_bdot + layer for layer in bdot_layers]
#ponowne przeiterowanie sie po katalogu bdotu i dodanie wskazanych wczesniej (w tablicy bdot_layers) warstw
for files in os.listdir(BDOT_path):
    czy_interesuje = files.startswith(bdot_layers_1[0]) or files.startswith(bdot_layers_1[1]) or files.startswith(bdot_layers_1[2]) or files.startswith(bdot_layers_1[3]) or files.startswith(bdot_layers_1[4]) or files.startswith(bdot_layers_1[5])
    if files.endswith('.shp') and czy_interesuje:
        print(files)
        path = BDOT_path +"\\" + files
        map.addDataFromPath(path)
    else:
        continue

PL.PZGiK.337.0210__OT_PTGN_A.shp
PL.PZGiK.337.0210__OT_PTKM_A.shp
PL.PZGiK.337.0210__OT_PTLZ_A.shp
PL.PZGiK.337.0210__OT_PTNZ_A.shp
PL.PZGiK.337.0210__OT_PTPL_A.shp
PL.PZGiK.337.0210__OT_PTRK_A.shp
PL.PZGiK.337.0210__OT_PTSO_A.shp
PL.PZGiK.337.0210__OT_PTTR_A.shp
PL.PZGiK.337.0210__OT_PTUT_A.shp
PL.PZGiK.337.0210__OT_PTWP_A.shp
PL.PZGiK.337.0210__OT_PTWZ_A.shp
PL.PZGiK.337.0210__OT_PTZB_A.shp
PL.PZGiK.337.0210__OT_ADMS_A.shp
PL.PZGiK.337.0210__OT_BUBD_A.shp
PL.PZGiK.337.0210__OT_PTLZ_A.shp
PL.PZGiK.337.0210__OT_PTWP_A.shp
PL.PZGiK.337.0210__OT_SKDR_L.shp
PL.PZGiK.337.0210__OT_SWRS_L.shp


In [13]:
#Przycinanie warstw
#wyselekcjonowanie konkretnej badanej gminy z wszystkich jednostek administracyjnych, zrobienie bufora, zapisanie warstwy i dodanie do projektu
jednostkiAdministracyjne = przedrostek_bdot + "ADMS_A"
granicaGminy = arcpy.conversion.FeatureClassToFeatureClass(jednostkiAdministracyjne, gdb_path , 'granicaGminy', "NAZWA = " + nazwa_gminy)
gmina_bufor = arcpy.analysis.Buffer(granicaGminy, wynikowe_path + "gminabuffer.shp", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")


arcpy.env.extent = wynikowe_path + "gminabuffer.shp"
map.addDataFromPath(wynikowe_path + "gminabuffer.shp")
#przyciecie pokrycia terenu do obszaru gminy i dodanie do projektu
arcpy.management.SelectLayerByLocation('pokrycieTerenu', "INTERSECT", granicaGminy) 
arcpy.management.CopyFeatures('pokrycieTerenu', wynikowe_path + "pokrycie.shp")
map.addDataFromPath(wynikowe_path + "pokrycie.shp")

In [14]:
#przyciecie, zapisanie i dodanie do projektu wszystkich warstw potrzebnych z bdota
def przytnij(warstwa):
    arcpy.management.SelectLayerByLocation(warstwa, "INTERSECT", gmina_bufor)
    if warstwa.endswith('BUBD_A'): #dla warstwy z budynkami sposrod tych w gminie wybiera jedynie mieszkalne
        arcpy.management.SelectLayerByAttribute(warstwa, "SUBSET_SELECTION", "FUNSZCZ LIKE 'budynekW%' or FUNSZCZ LIKE 'budynekJ%' ")
    arcpy.management.CopyFeatures(warstwa, wynikowe_path + warstwa[-6::] + ".shp")
    map.addDataFromPath(wynikowe_path + warstwa[-6::] + ".shp")

for warstwa in bdot_layers_1[1::]:
    przytnij(warstwa)

In [15]:
#usuniecie wczesniej dodanych warstw pokrycia terenu i xrodowych nieprzycietych warstw z bdota
for layer in map.listLayers():
    czy_interesuje = layer.name.startswith(bdot_layers_1[0]) or layer.name.startswith(bdot_layers_1[1]) or layer.name.startswith(bdot_layers_1[2]) or layer.name.startswith(bdot_layers_1[3]) or layer.name.startswith(bdot_layers_1[4]) or layer.name.startswith(bdot_layers_1[5])
    czy_PT = layer.name.startswith(przedrostek_bdot + "PT")
    if czy_interesuje or czy_PT:
        map.removeLayer(layer)
        print(layer)
    else:
        continue

In [16]:
#przypisanie odpowiednio przygotowanych i dodanych do projektu warstw do zmiennych
obszar_badania = "gminabuffer"
drogi = "SKDR_L"
woda_plynaca = "SWRS_L"
woda_stojaca = "PTWP_A"
budynki_mieszkalne = "BUBD_A"
las = 'PTLZ_A'
pokrycie = "pokrycie"

Kryterium 1 - odleglosc od konkurencji - >= 400 m

In [17]:
#utworzenie mapy odległości, następnie przycięcie jej do obszaru gminy, utworzenie rastra przydatności na podstawie funkcji liniowej
dist = arcpy.sa.EucDistance(hotele)
extractedByMask = arcpy.sa.ExtractByMask(dist, granicaGminy, "INSIDE");
Kryt1 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(400, 1000))
Kryt1.save(wynikowe_path + "kryt1.tif")
map.addDataFromPath(wynikowe_path + "kryt1.tif")

start
dist
extracted
fuzzied


Kryterium 2 - odległość od budynków mieszkalnych - 25-150 m 

In [18]:
#wykonujemy analogiczne kroki jak dla pierwszego kryterium
#jednak tworzymy dwa oddzielne rastry wynikowe dla dwóch funkcji liniowych - malejącej i rosnącej, nastepnie mnożymy je aby otrzymać ostateczny wynik
dist = arcpy.sa.EucDistance(budynki_mieszkalne)
extractedByMask = arcpy.sa.ExtractByMask(dist, granicaGminy, "INSIDE"); 
Kryt2_vol1 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(5, 25))
Kryt2_vol2 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(400,150))
Kryt2 = Kryt2_vol1*Kryt2_vol2
Kryt2.save(wynikowe_path + "kryt2.tif")
map.addDataFromPath(wynikowe_path + "kryt2.tif")

Kryterium 3 - odległość od istniejących dróg - 15 - 100 m

In [19]:
dist = arcpy.sa.EucDistance(drogi)
extractedByMask = arcpy.sa.ExtractByMask(dist, granicaGminy, "INSIDE"); 
Kryt3_vol1 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(0, 15))
Kryt3_vol2 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(350,100))
Kryt3 = Kryt3_vol1*Kryt3_vol2
Kryt3.save(wynikowe_path + "kryt3.tif")
map.addDataFromPath(wynikowe_path + "kryt3.tif")

Kryterium 4 - odległość od rzek i zbiorników wodnych >=20 m nieprzekraczalne do 20 m

In [20]:
#wyliczenie wartosci dla rzek i zbiornikow wodnych
dist = arcpy.sa.EucDistance(woda_stojaca)
dist2 = arcpy.sa.EucDistance(woda_plynaca)
extractedByMask = arcpy.sa.ExtractByMask(dist, granicaGminy, "INSIDE"); 
extractedByMask2 = arcpy.sa.ExtractByMask(dist2, granicaGminy, "INSIDE"); 
Kryt4_1_vol1 = extractedByMask>=20
Kryt4_1_vol2 = arcpy.sa.FuzzyMembership(extractedByMask, arcpy.sa.FuzzyLinear(350,100))
Kryt4_2_vol1 = extractedByMask2>=20
Kryt4_2_vol2 = arcpy.sa.FuzzyMembership(extractedByMask2, arcpy.sa.FuzzyLinear(350,100))
Kryt4_1 = Kryt4_1_vol1*Kryt4_1_vol2
Kryt4_2 = Kryt4_2_vol1*Kryt4_2_vol2
#poniewaz uwazam bliskosc zbiornikow jako szczegolny walor jako wynik dla tego kryterium biore wieksza wartosc z dwoch
#jesli jest blizej zbiornika wodnego to biore ta wartosc jako wynik, jesli blizej rzeki to wartosc wynikowa dla wody plynacej
Kryt4 = arcpy.ia.CellStatistics([Kryt4_1, Kryt4_2], "MAXIMUM", "DATA")
Kryt4.save(wynikowe_path + "kryt4.tif")
map.addDataFromPath(wynikowe_path + "kryt4.tif")
wody_ostr = Kryt4_1_vol1*Kryt4_2_vol1
wody_ostr.save(wynikowe_path + "wody_ostr.tif")
map.addDataFromPath(wynikowe_path + "wody_ostr.tif")

Kryterium 5 - pokrycie terenu - nie w lesie

In [22]:
# selekcja lasów i zagajników, nadanie im wartosci 0 w dodatkowycm polu, na podstawie ktorego pozniej tworzymy raster przydanosci
#nastepnie robimy reklasyfikacje, tak aby wszystkie pozostale tereny mialy wartosc 1
las_zagajnik = arcpy.management.SelectLayerByAttribute(las, "NEW_SELECTION", "X_KOD <> 'PTLZ03'", None) 
arcpy.management.CalculateField(las_zagajnik, "Value", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS") 
kryt_5 = arcpy.conversion.FeatureToRaster(las_zagajnik, "Value", "raster_lasy", 1) 
kryt_5 = arcpy.sa.Reclassify(kryt_5, "Value", "0 0;NODATA 1", "NODATA")
kryt_5 = arcpy.sa.ExtractByMask(kryt_5, granicaGminy, "INSIDE");
kryt_5.save(wynikowe_path + "kryt5.tif")
map.addDataFromPath(wynikowe_path + "kryt5.tif")

Kryterium 6 - nachylenie stoku maksymalnie 20%

In [23]:
slope = arcpy.sa.Slope(NMT, "PERCENT_RISE")
kryt_6 = arcpy.sa.FuzzyMembership(slope, arcpy.sa.FuzzyLinear(40, 5))
kryt_6 = arcpy.sa.ExtractByMask(kryt_6, granicaGminy, "INSIDE"); 
kryt_6.save(wynikowe_path + "kryt6.tif")
map.addDataFromPath(wynikowe_path + "kryt6.tif")

Kryterium 7 - dostep swiatla slonecznego - stoki poludniowe - SW - SE

In [24]:
aspect = arcpy.ddd.Aspect(NMT,'aspected')
aspected = arcpy.sa.ExtractByMask(aspect, granicaGminy, "INSIDE"); 
kryt_7_1 = arcpy.sa.FuzzyMembership(aspected, arcpy.sa.FuzzyLinear(45, 135))
kryt_7_2 = arcpy.sa.FuzzyMembership(aspected, arcpy.sa.FuzzyLinear(315, 225))
kryt_7 = kryt_7_1 * kryt_7_2
#przypisanie wartosci 1 dla terenow plaskich za pomoca funkcji Con
kryt_7 = arcpy.ia.Con(aspected, 1, kryt_7, "VALUE = -1")
kryt_7.save(wynikowe_path + "kryt7.tif")
map.addDataFromPath(wynikowe_path + "kryt7.tif")

Kryterium 8 - poza strefa ochronna gazociagu

In [25]:
dist = arcpy.sa.EucDistance(gazociag)
extractedByMask = arcpy.sa.ExtractByMask(dist, granicaGminy, "INSIDE");
Kryt8 = arcpy.sa.FuzzyMembership(dist, arcpy.sa.FuzzyLinear(25, 200))
Kryt8.save(wynikowe_path + "kryt8.tif")
gazo_ostr = extractedByMask > 25
gazo_ostr.save(wynikowe_path + "gazo_ostr.tif")
map.addDataFromPath(wynikowe_path + "kryt8.tif")
map.addDataFromPath(wynikowe_path + "gazo_ostr.tif")

Kryterium 9 - uzytkowanie terenu - RV, RVI, N

In [27]:
#przypisanie wartosci O dla poszczegolnych typow gleb i utworzenie na tej podstawie rastra przydatnosci
#analogicznie jak kryt 5
calculated = arcpy.management.CalculateField(gleby, "isGood", "fun(!TYP!)", "PYTHON3", """def fun(a):
    if a in(['G', 'E', 'M', 'T', 'F', 'Fb', 'Fc', 'FG']):
        return 0
    else:
        return 1""", "TEXT", "NO_ENFORCE_DOMAINS")
kryt_9 = arcpy.conversion.FeatureToRaster(gleby, 'isGood', wynikowe_path + "kryt_9.tif")
map.addDataFromPath(wynikowe_path + "kryt_9.tif")

Polaczenie kryteriów

In [28]:
#przemnożenie kryteriów ostrych
ostre = arcpy.sa.FuzzyOverlay([kryt_5, kryt_9, gazo_ostr, wody_ostr], 'AND')
ostre.save(wynikowe_path + "ostre.tif")

#połaczenie kryteriów rozmytych z uwzględnieniem wag
rozmyte_weighted = arcpy.sa.WeightedSum(arcpy.sa.WSTable([[Kryt1, "VALUE", 0.2], [Kryt2, "VALUE",0.05], [Kryt4, "VALUE",0.2],
									  [Kryt3, "VALUE", 0.15], [kryt_6, "VALUE",0.2], [kryt_7, "VALUE",0.1], [Kryt8, "VALUE",0.1]]))
rozmyte_weighted.save(wynikowe_path + "rozmyte_weighted.tif")
#połaczenie kryteriów rozmytych z przypisaniem im równych wag
rozmyte_equal = arcpy.sa.WeightedSum(arcpy.sa.WSTable([[Kryt1, "VALUE", (1/7)], [Kryt2, "VALUE",(1/7)], [Kryt4, "VALUE",(1/7)],
									  [Kryt3, "VALUE", (1/7)], [kryt_6, "VALUE",(1/7)], [kryt_7, "VALUE",(1/7)], [Kryt8, "VALUE",(1/7)]]))
rozmyte_equal.save(wynikowe_path + "rozmyte_equal.tif")

#w celu uzyskania ostatecznego wyniku przemnożenie wyników kryteriów rozmytych przez iloczyn kryteriów ostrych
wynik_equalWeights = rozmyte_equal*ostre
wynik_equalWeights.save(wynikowe_path + "wynik_equalWeights.tif")
map.addDataFromPath(wynikowe_path + "wynik_equalWeights.tif")
wynik_weighted = rozmyte_weighted*ostre
wynik_weighted.save(wynikowe_path + "wynik_weighted.tif")
map.addDataFromPath(wynikowe_path + "wynik_weighted.tif")

#sprawdzenie czy na rastrze wynikowym nie ma gdzies braku danych sprawdzic czy gdzies nie ma no data, tego nie chcemy, bo polaczony z jakakolwiek wartoscia daje brak danych
#czyNoData = arcpy.sa.Reclassify("wynik_weighted.tif", "VALUE", "0 0,946831 1;NODATA 0", "DATA");
#czyNoData.save(r"D:\OneDrive - Politechnika Warszawska\Anazlizy\Swieradow\swieradow\swieradow.gdb\czyNoData")

Przypisanie dzialek spelniajacych kryteria

In [46]:
#przyciecie dzialek z EGIBu powiatowego do obszaru gminy
dzialki_1 = arcpy.management.SelectLayerByLocation(dzialki, "WITHIN", granicaGminy)
dzialki_2 = arcpy.conversion.FeatureClassToShapefile(dzialki, wynikowe_path)
map.addDataFromPath(wynikowe_path + "dzialki.shp")

#utworzenie ostatecznego rastra przydatnosci, prog przydatnosci terenu 0.85
ostateczny_rowneWagi = wynik_equalWeights >0.85
ostateczny_wagi = wynik_weighted > 0.85
ostateczny_rowneWagi.save(wynikowe_path + "ostateczny_rowneWagi.tif")
ostateczny_wagi.save(wynikowe_path + "ostateczny_wagi.tif")

#zamiana rastrów przydatnosci na poligon i zaznaczenie jedynie tych które są przydatne jako działki pod hotel
rowne_polygon = arcpy.conversion.RasterToPolygon(ostateczny_rowneWagi, "rowne_polygon" ,"SIMPLIFY", "VALUE")
rowne_polygon = arcpy.management.SelectLayerByAttribute(rowne_polygon, "NEW_SELECTION", "gridcode = 1") 
wagi_polygon = arcpy.conversion.RasterToPolygon(ostateczny_wagi, "wagi_polygon" ,"SIMPLIFY", "VALUE")
wagi_polygon = arcpy.management.SelectLayerByAttribute(wagi_polygon, "NEW_SELECTION", "gridcode = 1") 

#stworzenie tabeli, która okresla ile dana dzialka ma procent wspolnej powierzchni z naszym wynikowymi obszarami
arcpy.analysis.TabulateIntersection(wynikowe_path + "dzialki.shp", "ID_DZIALKI", rowne_polygon, gdb_path +"\dzialki_TabulateIntersection", None, None, None, "UNKNOWN")
#polaczenie tabeli intersekcji z klasa dzialek, wybranie tych, ktorych powierzchnia w 70% sie pokrywa z terenem interesujacym
arcpy.management.AddJoin("dzialki", "ID_DZIALKI",gdb_path +"\dzialki_TabulateIntersection", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
dzialki_percentage = arcpy.management.SelectLayerByAttribute("dzialki", "NEW_SELECTION", "PERCENTAGE > 0.7") 
dzialki_percentage = arcpy.management.CopyFeatures("dzialki", "dzialki_percentage") 
#polaczenie sasiadujacych ze soba dzialek
dissolved_equal = arcpy.management.Dissolve(dzialki_percentage, wynikowe_path + "dzialki_per_dis.shp",  None, None,"SINGLE_PART")
map.addDataFromPath(wynikowe_path+ "dzialki_per_dis.shp")
#usuniecie polaczenia, zeby mozna bylo zrobic kolejne dla drugiego podejscia
arcpy.management.RemoveJoin("dzialki")
dissolved_equal = wynikowe_path + "dzialki_per_dis.shp"

#analogiczna procedura dla podejscia z roznymi wagami
arcpy.analysis.TabulateIntersection(wynikowe_path + "\dzialki.shp", "ID_DZIALKI", wagi_polygon, gdb_path+ "dzialki_TabulateIntersection_wag", None, None, None, "UNKNOWN")
arcpy.management.AddJoin("dzialki", "ID_DZIALKI",gdb_path+ "\dzialki_TabulateIntersection_wag", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
dzialki_percentage_wag = arcpy.management.SelectLayerByAttribute("dzialki", "NEW_SELECTION", "PERCENTAGE > 0.7") 
dzialki_percentage_wag = arcpy.management.CopyFeatures("dzialki", "dzialki_percentage_wag") 
dissolved_diff = arcpy.management.Dissolve(dzialki_percentage_wag, wynikowe_path + "dzialki_per_wag_dis.shp", None, None, "SINGLE_PART")
dissolved_diff = "dzialki_per_wag_dis"
map.addDataFromPath(wynikowe_path+ "dzialki_per_wag_dis.shp")

Kryterium 10 - obszar 1-6 ha

In [47]:
#dodaje kolumy geometrii dla spelniajacych warunki dzialek
#dodatkowo oprocz dodania do tabeli atrybutow powierzchi dodaje takze obwod, aby od razu przygotowac
#sobie pole w ktorym obliczam wspolczynnik zwartosci oszaru ktory wykorzytsam przy kolejnym kryterium
arcpy.management.CalculateGeometryAttributes(dissolved_equal, [["AREA", "AREA"], ["CIRCUIT", "PERIMETER_LENGTH"]])
arcpy.management.CalculateField(dissolved_equal, "compacness", "math.sqrt(!AREA!/((!CIRCUIT!*!CIRCUIT!)/(4*math.pi)))", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(dissolved_equal, "NEW_SELECTION", "AREA > 10000") 
dzialki_10_1 = arcpy.management.CopyFeatures(dissolved_equal, wynikowe_path + "dzialki_10_rowne.shp") 
dzialki_10_1 = "dzialki_10_rowne"
map.addDataFromPath(wynikowe_path+ "dzialki_10_rowne.shp")
#dla wyniku z wagami
arcpy.management.CalculateGeometryAttributes(dissolved_diff, [["AREA", "AREA"], ["CIRCUIT", "PERIMETER_LENGTH"]])
arcpy.management.CalculateField(dissolved_diff, "compacness", "math.sqrt(!AREA!/((!CIRCUIT!*!CIRCUIT!)/(4*math.pi)))", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(dissolved_diff, "NEW_SELECTION", "AREA > 10000") 
dzialki_10_2 = arcpy.management.CopyFeatures(dissolved_diff, wynikowe_path + "dzialki_10_wagi.shp") 
dzialki_10_2 = "dzialki_10_wagi"
map.addDataFromPath(wynikowe_path+ "dzialki_10_wagi.shp")

Kryterium 11 - wyselekcjonowanie najbardziej zwartych obszarów

In [48]:
#wybieram na podstawie wczesniej wyliczonego wspolczynnika dzialki wg mnie o odpowieniej zwartosci dla obu podejsc
arcpy.management.SelectLayerByAttribute(dzialki_10_1, "NEW_SELECTION", "compacness > 0.5") 
dzialki_11_1 = arcpy.management.CopyFeatures(dzialki_10_1, wynikowe_path + "wynik_zwartosc_rowne.shp") 
map.addDataFromPath(wynikowe_path+ "wynik_zwartosc_rowne.shp")

arcpy.management.SelectLayerByAttribute(dzialki_10_2, "NEW_SELECTION", "compacness > 0.5") 
dzialki_11_2 = arcpy.management.CopyFeatures(dzialki_10_2,  wynikowe_path +"wynik_zwartosc_wagi.shp")
map.addDataFromPath(wynikowe_path+ "wynik_zwartosc_wagi.shp")

Znalezienie najtańszego podłączenia do gazociągu

In [49]:
#utworzenie mapy kosztow wzglednych na podstawie polaczonej warstwy pokrycia terenu
#przypisanie kosztow do poszczegolnych rodzajow pokrycia
arcpy.management.CalculateField(pokrycie, "costs", "calc_pokrycie(!X_KOD!)", "PYTHON3", """def calc_pokrycie (x_kod):
    if x_kod in ('PTWP01', 'PTWP03', 'PTUT01', 'PTSO01','PTSO02', 'PTWZ01', 'PTWZ02' ):
        return 0
    elif x_kod in ('PTZB05','PTLZ02', 'PTLZ03', 'PTPL01'):
        return 50
    elif x_kod in ('PTZB01','PTZB04','PTZB03','PTKM02', 'PTWP02'):
        return 200
    elif x_kod in ('PTZB02', 'PTLZ01', 'PTUT03', 'PTKM01'):
        return 100
    elif x_kod in ('PTNZ02', 'PTNZ01', 'PTKM03'):
        return 150
    elif x_kod in ('PTTR02', 'PTGN01', 'PTGN02', 'PTGN03', 'PTGN04'):
        return 1
    elif x_kod in ('PTUT04', 'PTUT05', 'PTTR01'):
        return 20
    elif x_kod == 'PTUT02':
        return 90
    elif x_kod == 'PTRK01' or x_kod == 'PTRK02':
        return 1""", "FLOAT")
mapa_kosztow_wzgl = arcpy.conversion.PolygonToRaster(pokrycie, 'costs', wynikowe_path +"mapa_kosztow_wzgl.tif")
mapa_kosztow_wzgl = arcpy.sa.Reclassify(mapa_kosztow_wzgl, "Value", "0 NODATA", "DATA") 
mapa_kosztow_wzgl.save(wynikowe_path +"mapa_kosztow_wzgl_reclas.tif")

In [50]:
#mapa kosztow skumulowanych
cost_distance_raser_rowne = arcpy.sa.CostDistance(dzialki_10_1, mapa_kosztow_wzgl, None, wynikowe_path +"kierunki_v1.tif", None, None, None, None, ''); 
cost_distance_raser_rowne.save(wynikowe_path + "koszty_skumulowane.tif")
cost_distance_raser_wagi = arcpy.sa.CostDistance(dzialki_10_2, mapa_kosztow_wzgl, None,wynikowe_path +"kierunki_v2.tif", None, None, None, None, ''); 
cost_distance_raser_wagi.save(wynikowe_path + "koszty_skumulowane_wag.tif")

In [51]:
#znalezienie podlaczenia do gazociagu o najmniejszym koszcie i zamienienie na linie
cost_path_rowne = arcpy.sa.CostPath(gazociag, wynikowe_path + "koszty_skumulowane.tif",  wynikowe_path +"kierunki_v1.tif", "BEST_SINGLE", "Id", "INPUT_RANGE"); 
cost_path_wagi = arcpy.sa.CostPath(gazociag, wynikowe_path + "koszty_skumulowane_wag.tif", wynikowe_path +"kierunki_v2.tif", "BEST_SINGLE", "Id", "INPUT_RANGE"); 

arcpy.conversion.RasterToPolyline(cost_path_rowne, wynikowe_path + "podlaczenie_rowne.shp")
arcpy.conversion.RasterToPolyline(cost_path_wagi, wynikowe_path + "podlaczenie_wagi.shp")
map.addDataFromPath(wynikowe_path+   "podlaczenie_rowne.shp")
map.addDataFromPath(wynikowe_path + "podlaczenie_wagi.shp")